In [3]:
%load_ext dotenv
%dotenv
import os
from neo4j import GraphDatabase


AUTH = (os.getenv("NEO4J_USER"), os.getenv("NEO4J_PASSWORD"))

with GraphDatabase.driver(os.getenv("NEO4J_URL"), auth=(AUTH)) as driver:
    driver.verify_connectivity()

In [4]:
from importlib import reload
import router
import openai_assistant

reload(router)
reload(openai_assistant)

<module 'openai_assistant' from '/Users/oskarhane/Development/tmp/neo4j-genai-design/openai_assistant.py'>

In [6]:

from retrievers import plot_vector, movie, text2cypher
import builtin_retrievers
reload(plot_vector)
reload(movie)
reload(text2cypher)
reload(builtin_retrievers)

ass = openai_assistant.OpenAIAssitant()

answer_found_retriever = builtin_retrievers.AnswerInContext()
vector_retriever = plot_vector.PlotVectorRetriever(
    driver=driver,
    llm=ass
)
movie_retriever = movie.MovieRetriever(driver=driver)
text2cypher_retriever = text2cypher.Text2Cypher(driver=driver, llm=ass)

rag_app = router.RetrieverRouter(ass)
rag_app.load_retriever(answer_found_retriever)
rag_app.load_retriever(vector_retriever)
rag_app.load_retriever(movie_retriever)
rag_app.load_retriever(text2cypher_retriever)


In [7]:
response = rag_app.handle_input('What year was the movie "Casino" released?')
print("--- output ---")
print(response)

[{"type": "function", "function": {"name": "extract_answer_from_context", "description": "If a complete and satisfactionary answer to the question already is provided in context, use this tool.", "parameters": {"type": "object", "properties": {"answer": {"type": "string", "description": "The answer to the question"}}, "required": ["answer"]}}}, {"type": "function", "function": {"name": "get_movie_by_plot", "description": "Matches movie taglines / stories to a user's input, and returns information about the movie.", "parameters": {"type": "object", "properties": {"plot": {"type": "string", "description": "The plot / story / tagline of the movie"}}, "required": ["plot"]}}}, {"type": "function", "function": {"name": "get_movie_from_name", "description": "Get movie details by providing the movie title. Returns actors and plot.", "parameters": {"type": "object", "properties": {"title": {"type": "string", "description": "The title of the movie"}, "match_type": {"type": "string", "enum": ["ex

In [92]:
response = rag_app.handle_input('What movie is about two mobster best friends, and what year was it released? And who were some actors playing roles in it?')
print("--- output ---")
print(response)


[{"type": "function", "function": {"name": "extract_answer_from_context", "description": "If a complete and satisfactionary answer to the question already is provided in context, use this tool.", "parameters": {"type": "object", "properties": {"answer": {"type": "string", "description": "The answer to the question"}}, "required": ["answer"]}}}, {"type": "function", "function": {"name": "get_movie_by_plot", "description": "Matches movie taglines / stories to a user's input, and returns information about the movie.", "parameters": {"type": "object", "properties": {"plot": {"type": "string", "description": "The plot / story / tagline of the movie"}}, "required": ["plot"]}}}, {"type": "function", "function": {"name": "get_movie_from_name", "description": "Get movie details by providing the movie title. Returns actors and plot.", "parameters": {"type": "object", "properties": {"title": {"type": "string", "description": "The title of the movie"}, "match_type": {"type": "string", "enum": ["ex

In [52]:
response = rag_app.handle_input('How many actors were in the movie "Casino"')
print("--- output ---")
print(response)

[{"type": "function", "function": {"name": "extract_answer_from_context", "description": "If a complete and satisfactionary answer to the question already is provided in context, use this tool.", "parameters": {"type": "object", "properties": {"answer": {"type": "string", "description": "The answer to the question"}}, "required": ["answer"]}}}, {"type": "function", "function": {"name": "get_movie_by_plot", "description": "Matches movie taglines / stories to a user's input, and returns information about the movie.", "parameters": {"type": "object", "properties": {"plot": {"type": "string", "description": "The plot / story / tagline of the movie"}}, "required": ["plot"]}}}, {"type": "function", "function": {"name": "get_movie_from_name", "description": "Get movie details by providing the movie title. Returns actors and plot.", "parameters": {"type": "object", "properties": {"title": {"type": "string", "description": "The title of the movie"}, "match_type": {"type": "string", "enum": ["ex

In [96]:
response = rag_app.handle_input('What are two the highest rated movies Robert De Niro has acted in? What are the plots in those movies?')
print("--- output ---")
print(response)

[{"type": "function", "function": {"name": "extract_answer_from_context", "description": "If a complete and satisfactionary answer to the question already is provided in context, use this tool.", "parameters": {"type": "object", "properties": {"answer": {"type": "string", "description": "The answer to the question"}}, "required": ["answer"]}}}, {"type": "function", "function": {"name": "get_movie_by_plot", "description": "Matches movie taglines / stories to a user's input, and returns information about the movie.", "parameters": {"type": "object", "properties": {"plot": {"type": "string", "description": "The plot / story / tagline of the movie"}}, "required": ["plot"]}}}, {"type": "function", "function": {"name": "get_movie_from_name", "description": "Get movie details by providing the movie title. Returns actors and plot.", "parameters": {"type": "object", "properties": {"title": {"type": "string", "description": "The title of the movie"}, "match_type": {"type": "string", "enum": ["ex